In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000) ## 800GB?

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import branca

import shared_utils

from siuba import *
import pandas as pd

import datetime as dt
import time
from zoneinfo import ZoneInfo

import importlib

import gcsfs
fs = gcsfs.GCSFileSystem()

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

from IPython.display import display, Markdown


/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. Yo

In [2]:
pd.set_option("display.max_columns", 100)

In [3]:
analysis_date = dt.date(2022,5,25)

In [4]:
itp_id = 300

In [5]:
#route = 3479

In [6]:
df = (tbl.views.gtfs_schedule_fact_daily_trips()
 >>filter(_.calitp_itp_id==itp_id)
 >>filter(_.service_date == analysis_date)
 >>filter(_.is_in_service == True)
      >> collect()
)

In [7]:
df

,feed_key,trip_key,trip_id,route_id,calitp_itp_id,calitp_url_number,service_id,service_date,service_indicator,service_start_date,service_end_date,service_inclusion,service_exclusion,is_in_service,calitp_extracted_at,calitp_deleted_at,n_stops,n_stop_times,trip_first_departure_ts,trip_last_arrival_ts,service_hours
0,4100255323842349957,-8949615142462783434,883610,3501,300,0,77710,2022-05-25,0,2022-03-27,2022-08-13,True,None,True,2022-03-29,2099-01-01,14,14,64620,65820,0.333333
1,4100255323842349957,-2813523198932208659,882896,3491,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2099-01-01,16,16,42960,44100,0.316667
2,4100255323842349957,5755218391950692981,883675,3501,300,0,77710,2022-05-25,0,2022-03-27,2022-08-13,True,None,True,2022-03-29,2099-01-01,17,17,34380,35400,0.283333
3,4100255323842349957,-9040673325549959534,883665,3501,300,0,77710,2022-05-25,0,2022-03-27,2022-08-13,True,None,True,2022-03-29,2099-01-01,17,17,26580,27420,0.233333
4,4100255323842349957,7837556335134130732,882604,3489,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2099-01-01,17,17,50700,52320,0.450000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1511,4100255323842349957,2588888464705716502,881832,3485,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2099-01-01,53,53,49560,53686,1.146111
1512,4100255323842349957,-2781961165564786424,881849,3485,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2099-01-01,53,53,35280,38931,1.014167
1513,4100255323842349957,4301936633053868317,881876,3485,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2099-01-01,53,53,69720,73078,0.932778
1514,4100255323842349957,-2137808411563612856,881943,3485,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2099-01-01,54,54,71940,75060,0.866667


In [8]:
df.service_id.nunique()

6

In [9]:
df.service_id.value_counts()

10       1409
77710      52
40510      52
96910       1
12010       1
92610       1
Name: service_id, dtype: int64

In [10]:
(df
 >>filter(_.calitp_itp_id==itp_id)
 >>filter(_.service_date == analysis_date)
 >>filter(_.is_in_service == True)
 >> filter(_.calitp_extracted_at <= analysis_date, _.calitp_deleted_at >= analysis_date)
 >> summarize(unique= _.trip_id.nunique())
      
)

,unique
0,1516


In [11]:
(df
 >>filter(_.calitp_itp_id==itp_id)
 >>filter(_.service_date == analysis_date)
 >>filter(_.is_in_service == True)
 >> filter(_.calitp_extracted_at <= analysis_date, _.calitp_deleted_at >= analysis_date)
 >> group_by(_.route_id) 
 >> summarize(n=_.trip_id.nunique()) >> arrange(-_.n))

,route_id,n
0,3479,180
4,3485,153
8,3489,153
2,3481,145
9,3490,107
18,3501,104
12,3493,88
1,3480,82
15,3496,76
5,3486,71


In [12]:
rt = query_sql(f'''
SELECT * FROM `cal-itp-data-infra-staging.nla_views.test_rt_trips`
WHERE date = '{analysis_date}'
''')

/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/


In [13]:
rt

,calitp_itp_id,calitp_url_number,date,trip_id,vehicle_id
0,300,0,2022-05-25,880868,1300
1,300,0,2022-05-25,880880,1300
2,300,0,2022-05-25,880855,1300
3,300,0,2022-05-25,880788,1300
4,300,0,2022-05-25,880842,1300
...,...,...,...,...,...
1445,300,0,2022-05-25,881503,5319
1446,300,0,2022-05-25,883351,5320
1447,300,0,2022-05-25,881431,5320
1448,300,0,2022-05-25,881440,5320


In [14]:
df>>distinct(_.trip_id)

,trip_id
0,883610
1,882896
2,883675
3,883665
4,882604
...,...
1511,881832
1512,881849
1513,881876
1514,881943


In [15]:
df.sample()

,feed_key,trip_key,trip_id,route_id,calitp_itp_id,calitp_url_number,service_id,service_date,service_indicator,service_start_date,service_end_date,service_inclusion,service_exclusion,is_in_service,calitp_extracted_at,calitp_deleted_at,n_stops,n_stop_times,trip_first_departure_ts,trip_last_arrival_ts,service_hours
11,4100255323842349957,5308859412212180813,883409,3496,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2099-01-01,23,23,49500,53040,0.983333


In [16]:
len(df)

1516

In [17]:
join1 = df>>inner_join(_, rt, on = 'trip_id')

In [18]:
len(join1)

1450

In [19]:
join2 = (pd.merge(df, rt, how='outer', on='trip_id', indicator='have_rt'))

In [20]:
join2.have_rt.value_counts()

both          1450
left_only      168
right_only       0
Name: have_rt, dtype: int64

In [21]:
pct_ran = ('{:.1%}'.format((len(join2>>filter(_.have_rt=='left_only')))/(len(join2>>filter(_.have_rt=='both')))))

In [22]:
display(Markdown(f"On {analysis_date.strftime('%B %d')}, **{pct_ran}** "
                "of scheduled trips produced GTFS RT data"))

On May 25, **11.6%** of scheduled trips produced GTFS RT data

In [23]:
# checking the length of the string to see if there are any incomplete/null `trip_ids`

In [24]:
rt>>filter(_.trip_id.isnull())

,calitp_itp_id,calitp_url_number,date,trip_id,vehicle_id


In [25]:
rt['str_len'] = rt.trip_id.str.len()

In [26]:
rt>>arrange(-_.str_len)

,calitp_itp_id,calitp_url_number,date,trip_id,vehicle_id,str_len
0,300,0,2022-05-25,880868,1300,6
1,300,0,2022-05-25,880880,1300,6
2,300,0,2022-05-25,880855,1300,6
3,300,0,2022-05-25,880788,1300,6
4,300,0,2022-05-25,880842,1300,6
...,...,...,...,...,...,...
1445,300,0,2022-05-25,881503,5319,6
1446,300,0,2022-05-25,883351,5320,6
1447,300,0,2022-05-25,881431,5320,6
1448,300,0,2022-05-25,881440,5320,6


In [27]:
## Putting into a function

In [102]:
def find_pct_ran_trips(itp_id, analysis_date):
    gtfs_daily = (
        tbl.views.gtfs_schedule_fact_daily_trips()
        >> filter(_.calitp_itp_id == itp_id)
        >> filter(_.service_date == analysis_date)
        >> filter(_.is_in_service == True)
        >> collect()
    )
    rt = query_sql(
        f"""
        SELECT * FROM `cal-itp-data-infra-staging.nla_views.test_rt_trips`
        WHERE date = '{analysis_date}'
        """
    )
    
    rt['str_len'] = rt.trip_id.str.len()
    
    #joinging 
    join = (pd.merge(gtfs_daily, rt, how='outer', on='trip_id', indicator='have_rt'))
    
    return join

def get_pct_ran(join, date):
    ## eventually generate a dataframe with the columns (dates and pct_ran)
    day_pct_ran = {}
    day_pct_ran['date'] = date
    day_pct_ran['pct_trips_ran'] = (len(join>>filter(_.have_rt=='left_only')))/(len(join>>filter(_.have_rt=='both')))
    
    pct_ran = pd.DataFrame([day_pct_ran])

    return pct_ran


In [103]:
join = find_pct_ran_trips(itp_id, analysis_date)

/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/


In [104]:
join.head()

,feed_key,trip_key,trip_id,route_id,calitp_itp_id_x,calitp_url_number_x,service_id,service_date,service_indicator,service_start_date,service_end_date,service_inclusion,service_exclusion,is_in_service,calitp_extracted_at,calitp_deleted_at,n_stops,n_stop_times,trip_first_departure_ts,trip_last_arrival_ts,service_hours,calitp_itp_id_y,calitp_url_number_y,date,vehicle_id,str_len,have_rt
0,4100255323842349957,4599154915086895146,883636,3501,300,0,77710,2022-05-25,0,2022-03-27,2022-08-13,True,None,True,2022-03-29,2099-01-01,14,14,43680,44760,0.300000,NaN,NaN,NaN,NaN,NaN,left_only
1,4100255323842349957,8711124548360778187,883638,3501,300,0,77710,2022-05-25,0,2022-03-27,2022-08-13,True,None,True,2022-03-29,2099-01-01,14,14,45240,46320,0.300000,NaN,NaN,NaN,NaN,NaN,left_only
2,4100255323842349957,-8685835925528238184,882509,3489,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2099-01-01,14,14,43920,45240,0.366667,300.0,0.0,2022-05-25,2111,6.0,both
3,4100255323842349957,-4978263900171094743,882507,3489,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2099-01-01,14,14,42480,43800,0.366667,300.0,0.0,2022-05-25,2106,6.0,both
4,4100255323842349957,8161856840212366460,883623,3501,300,0,40510,2022-05-25,0,2022-03-27,2022-08-13,True,None,True,2022-03-29,2099-01-01,14,14,33540,34800,0.350000,NaN,NaN,NaN,NaN,NaN,left_only


In [105]:
get_pct_ran(join, analysis_date)

,date,pct_trips_ran
0,2022-05-25,0.115862


In [40]:
# add another date into the mix

In [35]:
analysis_date2 = dt.date(2022,5,26)

In [36]:
join2 = find_pct_ran_trips(itp_id, analysis_date2)

/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/


In [37]:
join2.head()

,feed_key,trip_key,trip_id,route_id,calitp_itp_id_x,calitp_url_number_x,service_id,service_date,service_indicator,service_start_date,service_end_date,service_inclusion,service_exclusion,is_in_service,calitp_extracted_at,calitp_deleted_at,n_stops,n_stop_times,trip_first_departure_ts,trip_last_arrival_ts,service_hours,calitp_itp_id_y,calitp_url_number_y,date,vehicle_id,str_len,have_rt
0,4100255323842349957,4599154915086895146,883636,3501,300,0,77710,2022-05-26,0,2022-03-27,2022-08-13,True,None,True,2022-03-29,2099-01-01,14,14,43680,44760,0.300000,NaN,NaN,NaN,NaN,NaN,left_only
1,4100255323842349957,-4606747982320804028,883651,3501,300,0,40510,2022-05-26,0,2022-03-27,2022-08-13,True,None,True,2022-03-29,2099-01-01,14,14,55380,56580,0.333333,NaN,NaN,NaN,NaN,NaN,left_only
2,4100255323842349957,-8596878389009704179,882903,3491,300,0,10,2022-05-26,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2099-01-01,16,16,67500,68640,0.316667,300.0,0.0,2022-05-26,1820,6.0,both
3,4100255323842349957,6053015981769901570,882597,3489,300,0,10,2022-05-26,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2099-01-01,17,17,45720,47220,0.416667,300.0,0.0,2022-05-26,1502,6.0,both
4,4100255323842349957,-5088714916844360309,883535,3498,300,0,10,2022-05-26,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2099-01-01,29,31,68400,69900,0.416667,300.0,0.0,2022-05-26,1807,6.0,both


In [38]:
join2.have_rt.value_counts()

both          1496
left_only      122
right_only       0
Name: have_rt, dtype: int64

In [100]:
get_pct_ran(join2, analysis_date2)

,date,pct_trips_ran
0,2022-05-26,0.081551


In [101]:
pct2

'8.2%'

In [ ]:
def get_pct_ran_df(date1_df, date2_df):
    ## eventually generate a dataframe with the columns (dates and pct_ran)
    for date in 
    pct_ran_date1
    pct_ran_date1 = ('{:.1%}'.format((len(date1_df>>filter(_.have_rt=='left_only')))/(len(date1_df>>filter(_.have_rt=='both')))))
    #return pct_ran


In [35]:
## other queries

In [12]:
# (tbl.views.gtfs_rt_fact_daily_feeds
#  >>filter(_.calitp_itp_id==itp_id)
#  >>filter(_.date==analysis_date)
# )

In [23]:
# trips =(tbl.views.gtfs_schedule_dim_trips()
#  >>filter(_.calitp_itp_id==itp_id) >> collect())


In [24]:
#trips

In [15]:
#pd.merge(df, trips, on="route_id")

In [16]:
# (tbl.views.gtfs_schedule_fact_daily_trips()
#  >>filter(_.calitp_itp_id==itp_id)
#  >>filter(_.service_date == analysis_date)
# )

In [17]:
# (
#     tbl.views.gtfs_rt_fact_daily_feeds()
#     >> filter(_.calitp_itp_id == itp_id)
#     >> arrange(_.date ==analysis_date)
# )

In [18]:
# airtable_organizations = (
#     tbl.airtable.california_transit_organizations()
#     >> select(_.itp_id, _.name, _.caltrans_district,
#               _.website, _.ntp_id, _.drmt_organization_name)
#     >> filter(_.itp_id == itp_id)
#     >> collect()
# )

In [19]:
# airtable_organizations